# 제4 고지 : 신경망 만들기 
## STEP 41 : 행렬의 곱

이번 단계의 주제는 '벡터의 내적' 과 '행렬의 곱' 이다. 

### 41.1 벡터의 내적과 행렬의 곱 

우선 벡터의 내적이란 무엇일까? 벡터 $\mathbf{a}=(a_1,\dots,a_n),\mathbf{b}=(b_1,\dots,b_n)$ 이 있다고 가정했을때, 다음과 같이 정의 된다. 
$$
\mathbf{ab}=a_1b_1+a_2b_2+\cdots+a_nb_n
$$

행렬의 곱은 다음과 같이 정의 된다. 
<p align='center'>
    <img src='../assets/그림 41-1.png' align='center' width='30%'>
</p>

이를 넘파이를 사용하여 구현하면 다음과 같다. 
```python
import numpy as np 
# 벡터의 내적
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.dot(a,b)
print(c)

# 행렬의 곱
a = np.array([[1,2][3,4]])
b = np.array([[5,6][7,8]])
c = np.dot(a,b)
print(c)

'''
32
[[19 22]
 [43 50]]
'''
```
### 41.2 행렬의 형상 체크 

<p align='center'>
    <img src='../assets/그림 41-2.png' align='center' width='30%'>
</p>

행렬과 벡터를 사용한 계산에서는 '형상' 에 주의해서 계산해야한다. 위의 그림과 같이 $\mathbf{a},\mathbf{b}$의 대응하는 차원이 동일해야 하며 그 결과는 $\mathbf{a}$의 행 차원, $\mathbf{b}$ 의 열 차원을 갖는 $\mathbf{c}$가 만들어진다.

### 41.3 행렬 곱의 역전파
행렬 곱의 역전파를 구현하기 앞서, 이론에 대해 살펴보고 `Matmul` 클래스와 `matmul` 함수로 구현한다.  
우선, $\mathbf{y=xW}$ 라는 계산을 예로 역전파를 살펴보자.

<p align='center'>
    <img src='../assets/그림 41-3.png' align='center' width='30%'>
</p>

우리가 구하고 싶은것은 최종적으로 **스칼라를 출력하는 계산** 이므로 스칼라 값이 $L$ 을 출력한다고 가정한다. 이때 $x_i$에 대한 미분 $\frac{\partial L}{\partial x_i}$는 다음과 같이 구할 수 있다.
$$
\frac{\partial L}{\partial x_i} = \sum_j\frac{\partial L}{\partial y_j}\frac{\partial y_j}{\partial x_i}
$$

여기서 $\frac{\partial L}{\partial x_i}$ 는 $x_i$를 변화시켰을때 $L$이 얼마나 변화하는지를 나타내는 '변화율' 이다. 즉, **$x_i$를 변화시키면  $\mathbf{y}$의 모든 원소가 변화** 한다. 그리고 $\mathbf{y}$의 각 원소의 변화를 통해 $L$이 변화하게 된다.  
따라서, **$x_i$ 에서 $L$ 에 이르는 연쇄법칙의 경로는 여러개** 있고, 그 총합이 $\frac{\partial L}{\partial x_i}$ 이다.

그런데 $\frac{\partial y_j}{\partial x_i}=W_{ij}$ 이므로, 위의 식을 다음과 같이 표현할 수 있다.
$$
\frac{\partial L}{\partial x_i} = \sum_j\frac{\partial L}{\partial y_j}W_{ij} 
$$
$$
\Rightarrow \frac{\partial L}{\partial \mathbf{x}}=\frac{\partial L}{\partial \mathbf{y}}\mathbf{W}^\top
$$
<span style='background-color : #ffdce0'>(참고, <b>$y_j = x_1W_{1j}+x_2W_{2j}+\cdots+x_HW_{Hj}$ 이므로 $\frac{\partial y_j}{\partial x_i}=W_{ij}$</b>)</span>

위의 식에서 알 수 있듯이, $\frac{\partial L}{\partial \mathbf{x}}$ 는 **행렬의 곱**으로 한번에 구할 수 있다.

<p align='center'>
    <img src='../assets/그림 41-4.png' align='center' width='30%'>
</p>

이를 바탕으로 역전파 수식을 도출해보면(단, 이번에는 $\mathbf{x}$의 형상이 $N\times D$), 다음과 같은 역전파 계산그래프를 그릴 수 있다.

<p align='center'>
    <img src='../assets/그림 41-5.png' align='center' width='30%'>
</p>

구체적으로 ,$\frac{\partial L}{\partial \mathbf{x}},\frac{\partial L}{\partial \mathbf{W}}$를 도출 해보면 다음과 같다.

<p align='center'>
    <img src='../assets/그림 41-6.png' align='center' width='30%'>
</p>

이제 DeZero로 구현하면 다음과 같다.
```python
class MatMul(Function):
    def forward(self, x, W):
        y = x.dot(W)
        return y

    def backward(self, gy):
        x, W = self.inputs
        gx = matmul(gy, W.T)
        gW = matmul(x.T, gy)
        return gx, gW


def matmul(x, W):
    return MatMul()(x, W)
```

In [4]:
import sys
sys.path.append("..")

import numpy as np 
import dezero.functions as F 
from dezero import Variable

x = Variable(np.random.randn(2,3))
W = Variable(np.random.randn(3,4))
y = F.matmul(x,W)
y.backward()

print(x.grad,x.grad.shape)
print(W.grad,W.grad.shape)


variable([[-1.66808072 -0.97788319  0.40978914]
          [-1.66808072 -0.97788319  0.40978914]]) (2, 3)
variable([[0.52159659 0.52159659 0.52159659 0.52159659]
          [0.54898049 0.54898049 0.54898049 0.54898049]
          [0.08680415 0.08680415 0.08680415 0.08680415]]) (3, 4)
